In [1]:
import boto3
import json
client = boto3.client('lookoutvision')

### Step1: Create a Project

In [2]:
response = client.create_project(
    ProjectName='MyLookOutforVision_Project',
    ClientToken='MyIdempotencyToken'
)
print(response)

{'ResponseMetadata': {'RequestId': 'd6ed6e8f-22cb-47c4-bdd7-eeda0b2caa34', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd6ed6e8f-22cb-47c4-bdd7-eeda0b2caa34', 'x-xss-protection': '1; mode=block', 'strict-transport-security': 'max-age=31540000; includeSubDomains', 'x-frame-options': 'DENY', 'x-content-type-options': 'nosniff', 'date': 'Tue, 26 Jan 2021 20:10:58 GMT', 'content-type': 'application/json', 'content-length': '196'}, 'RetryAttempts': 0}, 'ProjectMetadata': {'ProjectArn': 'arn:aws:lookoutvision:us-east-1:979749033494:project/MyLookOutforVision_Project', 'ProjectName': 'MyLookOutforVision_Project', 'CreationTimestamp': datetime.datetime(2021, 1, 26, 14, 37, 15, 113000, tzinfo=tzlocal())}}


### Step2: Create Training and Testing Dataset (seperately)

### Step3: Create Models

In [3]:
def create_model(project_name, output_bucket, output_folder):

    client=boto3.client('lookoutvision')

    try: 
        #Create a model
        print('Creating model...')

        output_config=dataset=json.loads('{ "S3Location": { "Bucket": "' + output_bucket + '", "Prefix": "'+ output_folder + '" } } ')

        response=client.create_model(ProjectName=project_name, OutputConfig=output_config)
        print('ARN: ' + response['ModelMetadata']['ModelArn'])
        print('Version: ' + response['ModelMetadata']['ModelVersion'])
        print('Status: ' + response['ModelMetadata']['Status'])
        print('Message: ' + response['ModelMetadata']['StatusMessage'])
        print('Done...')

    
    except Exception as e:
        print(e)
    
def main():
    project_name='MyLookOutforVision_Project'
    description='my project description'
    output_bucket = 'andwct-lookoutforvision'
    output_folder = 'output/'

    create_model(project_name, output_bucket, output_folder)



if __name__ == "__main__":
    main()

Creating model...
ARN: arn:aws:lookoutvision:us-east-1:979749033494:model/MyLookOutforVision_Project/1
Version: 1
Status: TRAINING
Message: The model is being trained.
Done...


### Step4: Viewing Performance Metrics

In [32]:
def describe_models(project_name):

    client=boto3.client('lookoutvision')

    try: 
        #list models
        response=client.list_models(ProjectName=project_name)
        print('Project: ' + project_name)
        for model in response['Models']:
            print('Version: ' + model['ModelVersion'])
            print('ARN: ' + model['ModelArn']) 
            if 'Description'  in model:
                print('Description: ' + model['Description']) 
            print('Status: ' + model['Status']) 
            print('Status Message: ' + model['StatusMessage']) 

            # get model description
            model_description=client.describe_model(ProjectName=project_name, ModelVersion=model['ModelVersion'])
            print('Status: ' + model_description['ModelDescription']['Status'])
            print('Message: ' + model_description['ModelDescription']['StatusMessage'])
            
            if 'Performance' in model_description['ModelDescription']:
                print('Recall: ' + str(model_description['ModelDescription']['Performance']['Recall']))
                print('Precision: ' + str(model_description['ModelDescription']['Performance']['Precision']))
            
            if 'OutputConfig' in model_description['ModelDescription']:
                print('Output config: ' + str(model_description['ModelDescription']['OutputConfig']))
            print()

        print('Done...')


    except Exception as e:
        print(e)
    
def main():
    project_name='MyLookOutforVision_Project'
    describe_models(project_name)


main()

Project: MyLookOutforVision_Project
Version: 1
ARN: arn:aws:lookoutvision:us-east-1:979749033494:model/MyLookOutforVision_Project/1
Status: TRAINED
Status Message: Training completed successfully.
Status: TRAINED
Message: Training completed successfully.
Recall: 1.0
Precision: 0.875
Output config: {'S3Location': {'Bucket': 'andwct-lookoutforvision', 'Prefix': 'output/'}}

Done...


### Step5: Detection Anomalies in Action

In [37]:
!pip install wget
!wget s3://andwct-lookoutforvision/circuit_board/extra_images/ 

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=afaf4006843b2875275f6b902841d742d69a5a53bac0af5af65c16d8c4c4f02a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/90/1d/93/c863ee832230df5cfc25ca497b3e88e0ee3ea9e44adc46ac62
Successfully built wget
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
s3://andwct-lookoutforvision/circuit_board/extra_images/: Unsupported scheme ‘s3’.


In [ ]:
def detect_anomalies(project_name,model_version,photo):
     

    client=boto3.client('lookoutvision')

    #Call DetectAnomalies 

    with open(photo, 'rb') as image:
        response = client.detect_anomalies(ProjectName=project_name, 
        ContentType='image/jpeg',
        Body=image.read(),
        ModelVersion=model_version)
    print ('Anomalous?: ' + str(response['DetectAnomalyResult']['IsAnomalous']))
    print ('Confidence: ' + str(response['DetectAnomalyResult']['Confidence']))


def main():

    project_name='MyLookOutforVision_Project'
    photo='image.jpg'
    model_version='1'
  
    anomalous=detect_anomalies(project_name,model_version,photo)
    
main()